<a href="https://colab.research.google.com/github/ColinJ69/PocketDerm/blob/main/ML/hair_color_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import zipfile
import os

In [ ]:
import torch
from torch import nn
from torchvision import transforms, datasets
import torchvision
from torch.utils.data import DataLoader

In [ ]:
data_path = Path('hair_color')

if not data_path.is_dir():
    data_path.mkdir(exist_ok =True, parents=True)
if os.path.exists('hair_color.zip'):
  with zipfile.ZipFile("hair_color.zip", "r") as zip_ref:
    zip_ref.extractall(data_path)


In [ ]:
transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((640,640)), torchvision.transforms.ToTensor()])
image_folder = datasets.ImageFolder(root=data_path, transform=transforms,target_transform=None)
image_dataloader = DataLoader(image_folder, num_workers=2, batch_size=8, shuffle=True)
class_names = image_folder.classes

['Dark', 'Fair', 'Tan']


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
model = resnet50(weights=ResNet50_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


In [ ]:
from tqdm.auto import tqdm

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.7)
def train():
  epochs = 20
  model.train()
  for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    for batch, (X,y) in enumerate(image_dataloader):
      optimizer.zero_grad()
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      loss.backward()
      optimizer.step()
      epoch_loss += loss
    epoch_loss /= len(image_dataloader)
    print(f' LOSS: {epoch_loss}')



In [ ]:
train()

  0%|          | 0/16 [00:00<?, ?it/s]

 LOSS: 6.185319423675537
 LOSS: 3.218360185623169
 LOSS: 1.3094704151153564
 LOSS: 0.7682825922966003
 LOSS: 0.6388211250305176
 LOSS: 0.5186536908149719
 LOSS: 0.47205856442451477
 LOSS: 0.3669064939022064
 LOSS: 0.316888689994812
 LOSS: 0.23461481928825378
 LOSS: 0.276470422744751
 LOSS: 0.1939830332994461
 LOSS: 0.1673509031534195
 LOSS: 0.1587321162223816
 LOSS: 0.16200515627861023
 LOSS: 0.13929510116577148


In [ ]:
torch.save(model, 'hair_color_model2.pth')